In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np
import collections
import math
import os
import random
from six.moves import urllib
import io
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output, Image, display, HTML
import tensorflow as tf
import tensorflow_hub as hub
import sklearn.metrics as sk_metrics
import time
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten
from tensorflow.keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import RMSprop,Adam
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
## set train, validation, and test paths
train_path = '../input/dataset2/dataset70-10-20/train/Fruit'
validation_path = '../input/dataset2/dataset70-10-20/val/Fruit'
test_path = '../input/dataset2/dataset70-10-20/test/Fruit'

In [ ]:
packages_classes=[]
for root, subdirectories, files in os.walk(train_path):
    for subdirectory in subdirectories:
        packages_classes.append(subdirectory)
print(len(packages_classes))

In [ ]:
train_batches  = ImageDataGenerator().flow_from_directory(
    train_path, target_size=(224, 224), classes = packages_classes, batch_size = 50)

validation_batches  = ImageDataGenerator().flow_from_directory(
    validation_path, target_size=(224, 224), classes = packages_classes, batch_size = 10)

test_batches  = ImageDataGenerator().flow_from_directory(
    test_path, target_size=(224, 224), classes = packages_classes, batch_size = 10)

In [ ]:
#take a look at output of the generators
for data_batch, labels_batch in train_batches:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

## Data Augmentation

In [ ]:
## Augmentation datagenerator
train_datagen_aug = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

## train generator
train_generator_aug = train_datagen_aug.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=25,
        class_mode='categorical')

## test generator, required for prediction
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=1,
        shuffle = False,
        class_mode='categorical')

## validation generator
val_generator = test_datagen.flow_from_directory(
        validation_path,
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

In [ ]:
## set steps per epoch for train
train_filenames = train_generator_aug.filenames
steps_train = len(train_filenames)/train_generator_aug.batch_size

## set steps per epoch for validation
val_filenames = val_generator.filenames
steps_val = len(val_filenames)/val_generator.batch_size

## test
filenames_test = test_generator.filenames
nb_samples_test= len(filenames_test)
true_classes = test_generator.classes
target_names = packages_classes

# Xception

In [ ]:

from keras.applications import xception
model = xception.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3),classes=28,pooling='avg')
print (model.summary())

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
count = 0
for layer in model.layers:
    count += 1
count

In [ ]:
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-10:]:
    layer.trainable = True

In [ ]:
x=model.output
x=tf.keras.layers.Dropout(.4)(x)
output=Dense(28,activation='softmax')(x)
New_Model=Model(model.input,output)

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01,patience= 3, verbose=1)
mc = ModelCheckpoint("./vgg19_best2.h5", monitor='val_accuracy', verbose=1, save_best_only=True)
cb=[es, mc]
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_path,
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

## set steps per epoch for train
train_filenames = train_generator.filenames
steps_train = len(train_filenames)/train_generator.batch_size

## set steps per epoch for validation
validation_filenames = validation_generator.filenames
steps_valid = len(validation_filenames)/validation_generator.batch_size

New_Model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(1e-4,epsilon=1e-07),
              metrics=['acc'])

fit_generator_3 = New_Model.fit(
        train_generator,
        steps_per_epoch=steps_train,
        epochs=200,
        validation_data=validation_generator,
        validation_steps=steps_valid,
        callbacks=[cb]
)

In [ ]:
acc = New_Model.history['acc']
val_acc = New_Model.history['val_acc']
loss = New_Model.history['loss']
val_loss = New_Model.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
translearn3_vgg16 = Sequential()
translearn3_vgg16.add(base_model_vgg16)
#translearn3_vgg16.add(Dense(512, activation = 'relu'))
translearn3_vgg16.add(GlobalAveragePooling2D())
translearn3_vgg16.add(Dropout(0.3))
translearn3_vgg16.add(Dense(31, activation='softmax'))
translearn3_vgg16.summary()

## compile model
translearn3_vgg16.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

# Callbacks
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01,patience= 4, verbose=1)
mc = ModelCheckpoint("vgg16_best3.h5", monitor='val_accuracy', verbose=1, save_best_only=True)
cb=[es, mc]

## fit model
model3_vgg16 = translearn3_vgg16.fit(
        train_generator_aug,
        steps_per_epoch=steps_train,
        epochs=50,
        validation_data=val_generator,
        validation_steps=steps_val,
        verbose=1,
        callbacks=cb)

In [ ]:
acc = model2_vgg16.history['acc']
val_acc = model2_vgg16.history['val_acc']
loss = model2_vgg16.history['loss']
val_loss = model2_vgg16.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Predict

In [ ]:
New_Model.save("fruits_xeption.h5")

In [ ]:
model = models.load_model("fruits_xeption.h5")

In [ ]:
vgg16_pred

In [ ]:
vgg16_pred= model.predict(test_generator, steps = nb_samples_test, verbose=2)
vgg16_y_pred1 = vgg16_pred.argmax(axis=1)

### Confusion Matrix

In [ ]:
vgg16_CM1 = confusion_matrix(vgg16_y_pred1, true_classes)
ax= plt.subplot()
sns.heatmap(vgg16_CM1, annot=True, fmt='g', ax=ax)

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels') 
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['Fruit', 'Packages', 'Vegetables']) 
ax.yaxis.set_ticklabels(['Fruit', 'Packages', 'Vegetables'])

### Classification Report

In [ ]:
print(classification_report(true_classes, vgg16_y_pred1, target_names=target_names))